In [2]:
import gensim
from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/data/GoogleNews-vectors-negative300.bin.gz", binary=True, limit=1000000)
wv.save_word2vec_format('vectors.csv')

In [4]:
import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv('/content/encoded-phrases (1).csv')

In [13]:
df.head()

,Phrases
0,how company compares to its peers?
1,what is the detailed income statement breakdow...
2,world premium penetration in 2020
3,How does the forecasted insurance premium pene...
4,what are the total losses for companies in cou...


In [19]:
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
import pandas as pd
import gensim
from gensim.models import KeyedVectors
import nltk
from nltk.tokenize import word_tokenize

# Load the pre-trained Word2Vec model
# model_path = "/content/drive/MyDrive/data/GoogleNews-vectors-negative300.bin.gz"
# wv = KeyedVectors.load_word2vec_format(model_path, binary=True, limit=1000000)

# Read the CSV file containing phrases (update the file path)
csv_file = "/content/encoded-phrases (1).csv"
df = pd.read_csv(csv_file)

# Tokenize the phrases into words
nltk.download('punkt')  # Ensure NLTK's tokenizer is available
df['tokenized_phrases'] = df['Phrases'].apply(lambda x: word_tokenize(x))

# Function to get Word2Vec embeddings for words in a phrase
def get_word_embeddings(phrase):
    word_embeddings = []
    for word in phrase:
        if word in wv:
            word_embeddings.append(wv[word])
    return word_embeddings

# Apply the function to get word embeddings for each phrase
df['word_embeddings'] = df['tokenized_phrases'].apply(get_word_embeddings)

# Now, 'df' contains the original phrases, tokenized phrases, and Word2Vec embeddings for each word in the phrases.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
df.head()

,Phrases,tokenized_phrases,word_embeddings
0,how company compares to its peers?,"[how, company, compares, to, its, peers, ?]","[[0.26953125, 0.0859375, 0.09423828, 0.0410156..."
1,what is the detailed income statement breakdow...,"[what, is, the, detailed, income, statement, b...","[[0.13964844, -0.006164551, 0.21484375, 0.0727..."
2,world premium penetration in 2020,"[world, premium, penetration, in, 2020]","[[-0.063964844, 0.068359375, 0.22460938, 0.131..."
3,How does the forecasted insurance premium pene...,"[How, does, the, forecasted, insurance, premiu...","[[0.16015625, 0.21679688, 0.05493164, 0.207031..."
4,what are the total losses for companies in cou...,"[what, are, the, total, losses, for, companies...","[[0.13964844, -0.006164551, 0.21484375, 0.0727..."


In [34]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import cosine_similarity

# Read the CSV file containing phrases
csv_file = "/content/encoded-phrases (1).csv"  # Update the file path
df = pd.read_csv(csv_file)

# Function to calculate L2 distance between two phrase embeddings
def l2_distance(emb1, emb2):
    return np.linalg.norm(emb1 - emb2)

# Function to calculate Cosine distance between two phrase embeddings
def cosine_distance(emb1, emb2):
    return 1 - cosine_similarity([emb1], [emb2])[0][0]

# Calculate the normalized sum of individual word token embeddings for each phrase
def get_normalized_phrase_embeddings(tokenized_phrase, wv):
    # Filter out words that are not in the Word2Vec model
    valid_words = [word for word in tokenized_phrase if word in wv]

    if not valid_words:
        return None

    # Sum the word embeddings
    phrase_embedding = np.sum([wv[word] for word in valid_words], axis=0)

    # Normalize the phrase embedding
    phrase_embedding /= np.linalg.norm(phrase_embedding)

    return phrase_embedding

# Create a dictionary to store normalized phrase embeddings
phrase_embeddings = {}
for index, row in df.iterrows():
    phrase = row['Phrases']  # Assuming the phrase column is named 'phrase'
    tokenized_phrase = word_tokenize(phrase)  # Tokenize the phrase
    embedding = get_normalized_phrase_embeddings(tokenized_phrase, wv)
    if embedding is not None:
        phrase_embeddings[index] = embedding

# Calculate L2 and Cosine distances between phrases and store results
num_phrases = len(phrase_embeddings)
l2_distances = np.zeros((num_phrases, num_phrases))
cosine_distances = np.zeros((num_phrases, num_phrases))

for i in range(num_phrases):
    for j in range(i + 1, num_phrases):
        emb1 = phrase_embeddings[i]
        emb2 = phrase_embeddings[j]

        # Calculate and store L2 and Cosine distances
        l2_distances[i, j] = l2_distance(emb1, emb2)
        l2_distances[j, i] = l2_distances[i, j]  # Make the matrix symmetric
        cosine_distances[i, j] = cosine_distance(emb1, emb2)
        cosine_distances[j, i] = cosine_distances[i, j]  # Make the matrix symmetric

# Now, you have matrices 'l2_distances' and 'cosine_distances' where
# l2_distances[i, j] and cosine_distances[i, j] represent the distances
# between the i-th and j-th phrases.


In [36]:
cosine_distances.shape

(50, 50)

In [40]:
cosine_distances[1]

array([0.52838105, 0.        , 0.65984461, 0.49637181, 0.43356353,
       0.57172257, 0.49927229, 0.53610048, 0.37797779, 0.57616723,
       0.45142323, 0.53357357, 0.31594247, 0.64775869, 0.46085513,
       0.5056521 , 0.49799502, 0.48204654, 0.5072844 , 0.4184978 ,
       0.49181515, 0.56745639, 0.41501015, 0.45148534, 0.48339802,
       0.46090108, 0.65579587, 0.67132181, 0.93007269, 0.63598031,
       0.67289245, 0.60075361, 0.53747037, 0.72700062, 0.68523699,
       0.58424681, 0.55214977, 0.67855191, 0.58264938, 0.55338442,
       0.51723769, 0.50718817, 0.46753633, 0.56204906, 0.62648931,
       0.50283551, 0.54172164, 0.5299044 , 0.55130973, 0.56968066])

In [72]:
import pandas as pd
import numpy as np
from scipy.spatial import distance

# Read the CSV file containing phrases
csv_file = "/content/encoded-phrases (1).csv"  # Update the file path
df = pd.read_csv(csv_file)

# Load precomputed Cosine distances between phrases (from previous calculations)
# The variable 'cosine_distances' is assumed to contain these distances

# Function to calculate the normalized sum of individual word token embeddings for a phrase
def get_normalized_phrase_embedding(phrase, wv):
    tokens = phrase.split()  # Split the phrase into words
    valid_tokens = [token for token in tokens if token in wv.key_to_index]  # Filter valid tokens
    if valid_tokens:
        # Calculate the normalized sum of word embeddings
        word_embeddings = [wv[token] for token in valid_tokens]
        phrase_embedding = np.sum(word_embeddings, axis=0)
        phrase_embedding /= np.linalg.norm(phrase_embedding)
        return phrase_embedding
    else:
        return None

# Create a list to store normalized phrase embeddings
phrase_embeddings = [get_normalized_phrase_embedding(phrase, wv) for phrase in df['Phrases']]

# Function to find the closest match for a given input phrase
def find_closest_match(input_phrase, phrase_embeddings, df):
    input_embedding = get_normalized_phrase_embedding(input_phrase, wv)

    if input_embedding is not None:
        # Calculate Cosine distances between the input phrase and all other phrases
        distances = [distance.cosine(input_embedding, emb) for emb in phrase_embeddings]

        # Find the index of the closest match
        closest_index = np.argmin(distances)

        # Get the closest phrase and its distance
        closest_phrase = df['Phrases'][closest_index]
        closest_distance = distances[closest_index]

        return closest_phrase, closest_distance
    else:
        return "Input phrase has no valid tokens.", None




In [53]:
def find_closest_match_from_user_input(s):
    closest_phrase, distance = find_closest_match(s, phrase_embeddings, df)

    if closest_phrase is not None:
        print(f"Closest phrase: {closest_phrase}")
        print(f"Distance: {distance}")
    else:
        print("No valid tokens found in the input phrase.")



In [54]:
for i in range(len(df['Phrases'])):
  find_closest_match_from_user_input(df['Phrases'][i])

Closest phrase: how company compares to its peers?
Distance: 0
Closest phrase: what is the detailed income statement breakdown of Axa?
Distance: 0
Closest phrase: world premium penetration in 2020
Distance: 0
Closest phrase: How does the forecasted insurance premium penetration in country trend compare to its peers?
Distance: 0
Closest phrase: what are the total losses for companies in country  non life market?
Distance: 0
Closest phrase: Insurance premiums market in Country
Distance: 0
Closest phrase: How have the profit margins been for the airlines industry through the years, by regions?
Distance: 0
Closest phrase: What is the complete profile of the top oil rigs?
Distance: 0
Closest phrase: what is company general information?
Distance: 0
Closest phrase: How is the airline industry performing globally?
Distance: 0
Closest phrase: how does economic profit for in country compare to others?
Distance: 0
Closest phrase: most profitable insurance company India
Distance: 0
Closest phrase:

In [73]:
# Example usage:
user_input = "Your user input phrase goes here"
closest_phrase, distance = find_closest_match(user_input, phrase_embeddings, df)

if closest_phrase is not None:
    print(f"Closest phrase: {closest_phrase}")
    print(f"Distance: {distance}")
else:
    print("No valid tokens found in the input phrase.")

Closest phrase: What is the share price of Qantas?
Distance: 0.5419219732284546
